In [1]:
from matplotlib import pyplot as plt
import xarray as xr
import numpy as np
import dask_jobqueue
import dask
from distributed import Client
from dask.diagnostics import progress
from tqdm.autonotebook import tqdm
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
import intake
import fsspec
#import seaborn as sns
import gcsfs
import cftime
import pandas as pd
import xesmf as xe

/tmp/ipykernel_1734175/4085156963.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
################################
savefigs   = '/global/scratch/users/harsha/savefigs/Feb21/'
cesm2_path = '/global/scratch/users/harsha/LENS/cesm2/tasmax/'
cvals      = '/global/scratch/users/harsha/LENS/cesm2/cvals/detrended/'
############
pi_year  = 1865
eoc_year = 2085

In [3]:
job_extra = ['--qos=cf_lowprio','--account=ac_cumulus'] 
#job_extra =['--qos=lr6_lowprio','--account=ac_cumulus','--constraint=lr6_m192']
#job_extra =['--qos=condo_cumulus_lr6','--account=lr_cumulus','--constraint=lr6_m192']
#job_extra =['--qos=lr_lowprio','--account=ac_cumulus']
cluster = dask_jobqueue.SLURMCluster(queue="cf1", cores=10, walltime='5:00:00', 
                local_directory='/global/scratch/users/harsha/dask_space/', 
                log_directory='/global/scratch/users/harsha/dask_space/', 
                job_extra_directives=job_extra, interface='eth0', memory="192GB") 
client  = Client(cluster) 
cluster.scale_up(3)

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40303 instead
  warnings.warn(


In [4]:
cluster

Dashboard: http://10.0.39.8:40303/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.39.8:41703,Workers: 0
Dashboard: http://10.0.39.8:40303/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
chicago_lat=41.88
chicago_lon=(360-87.6298)%360
# #
# cvals_ = '/Users/hrh/Desktop/TwoMoments21/cvals/cmip6/'

In [6]:
# calculate global means
def get_lat_name(ds):
    for lat_name in ['lat', 'latitude']:
        if lat_name in ds.coords:
            return lat_name
    raise RuntimeError("Couldn't find a latitude coordinate")

def global_mean(ds):
    lat = ds[get_lat_name(ds)]
    weight = np.cos(np.deg2rad(lat))
    weight /= weight.mean()
    other_dims = set(ds.dims) - {'quantile'}
    return (ds * weight).mean(other_dims)

def detrend_data(ds, central_year):
    # Assumes that the ds has coordinates day, year and member.
    
    #Fit a linear fuction and extract slope
    pcoeffs = ds.polyfit(dim='year',deg=1)
    slope   = pcoeffs.polyfit_coefficients.sel(degree=1)
    
    #Calculate trend
    ds_trend   = slope*(ds['year']- central_year)
    
    #Detrend by subtracting the trend from the data
    ds_detrended = ds  - ds_trend
    
    return ds_detrended

In [7]:
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")
col

,unique
activity_id,18
institution_id,36
source_id,88
experiment_id,170
member_id,657
table_id,37
variable_id,700
grid_label,10
zstore,514818
dcpp_init_year,60


In [8]:
# 2. Search for maximum temperature for son 30
expts = ['ssp370','historical']

cat = col.search(
    experiment_id=expts,
    table_id='day',
    variable_id='tasmax',
    #grid_label='gn'
)

query = dict(
    experiment_id=expts,
    table_id='day',
    variable_id=['tasmax'],
    member_id = 'r1i1p1f1',
)

col_subset = col.search(require_all_on=["source_id"], **query)
col_subset.df.groupby("source_id")[
    ["experiment_id", "variable_id", "table_id","member_id"]
].nunique()

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/intake_esm/_search.py:82: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for _, group in grouped:


,experiment_id,variable_id,table_id,member_id
source_id,,,,
ACCESS-CM2,2,1,1,1
AWI-CM-1-1-MR,2,1,1,1
BCC-ESM1,2,1,1,1
CMCC-ESM2,2,1,1,1
CanESM5,2,1,1,1
EC-Earth3,2,1,1,1
EC-Earth3-AerChem,2,1,1,1
EC-Earth3-Veg-LR,2,1,1,1
FGOALS-g3,2,1,1,1


In [9]:
df = col_subset.df
model_counts = df.groupby('source_id').size()
print(model_counts)

source_id
ACCESS-CM2           2
AWI-CM-1-1-MR        2
BCC-ESM1             2
CMCC-ESM2            2
CanESM5              2
EC-Earth3            2
EC-Earth3-AerChem    2
EC-Earth3-Veg-LR     2
FGOALS-g3            2
GFDL-ESM4            2
INM-CM4-8            2
INM-CM5-0            2
IPSL-CM6A-LR         2
KACE-1-0-G           2
MIROC6               2
MPI-ESM-1-2-HAM      2
MPI-ESM1-2-HR        2
MPI-ESM1-2-LR        2
MRI-ESM2-0           2
NorESM2-LM           2
NorESM2-MM           2
dtype: int64


In [10]:
# def drop_all_bounds(ds):
#     drop_vars = [vname for vname in ds.coords
#                  if (('_bounds') in vname ) or ('_bnds') in vname]
#     return ds.drop(drop_vars)

# def open_dset(df):
#     assert len(df) == 1
#     ds = xr.open_zarr(fsspec.get_mapper(df.zstore.values[0]), consolidated=True)
#     return drop_all_bounds(ds)

# def open_delayed(df):
#     return dask.delayed(open_dset)(df)

# from collections import defaultdict
# dsets = defaultdict(dict)

# for group, df in col_subset.df.groupby(by=['source_id', 'experiment_id']):
#     dsets[group[0]][group[1]] = open_delayed(df)

In [11]:
# %%time
# # Trigger computation
# dsets_ = dask.compute(dict(dsets))[0]

In [12]:
#Define coarse grid to regrid on
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-90, 91, 1.5)),
                     'lon': (['lon'], np.arange(0, 361, 1.5))})

In [13]:
def drop_feb29(ds):
    # Check if the dataset's calendar is not '360_day'
    calendar = ds.time.encoding.get('calendar', None)
    print(ds.attrs['source_id'],calendar)
    if calendar != '360_day':
        ds = ds.convert_calendar('365_day')
    return ds


def to_daily(ds):
    # Check and deal with different datetime types
    if isinstance(ds['time'].values[0], np.datetime64):
        pass
    elif isinstance(ds['time'].values[0], cftime.datetime):
        pass
    else:
        # convert time coordinate to datetime64 objects
        ds['time'] = ds['time'].astype('datetime64[ns]')
    year      = ds.time.dt.year
    dayofyear = ds.time.dt.dayofyear

    # assign new coords
    ds = ds.assign_coords(year=("time", year.data), dayofyear=("time", dayofyear.data))

    # reshape the array to (..., "day", "year")
    return ds.set_index(time=("year", "dayofyear")).unstack("time")  


def extract_data(ds):
    """
    Extract data for djf from the dataset 'ds' for specific time and spatial range.

    Parameters:
    - ds (xarray.Dataset): Input dataset

    Returns:
    - xarray.Dataset: Dataset subsetted for djf and the specified space and time range.
    """    

    subset1 = ds.sel(year=slice(1850, 1879))
    subset2 = ds.sel(year=slice(2071, 2100))
    
    subset = xr.concat([subset1, subset2], dim='year')  

    return subset

def is_leap(year):
    """Check if a year is a leap year."""
    return (year % 4 == 0) and ((year % 100 != 0) or (year % 400 == 0))


In [14]:
quants = np.linspace(0,1.0,30)

def compute_quantiles(ds, quantiles=quants):
    return ds.chunk(dict(year=-1)).quantile(quantiles, dim='year',skipna=False)

def regrid(ds, ds_out):
    experiment_id = ds.attrs['experiment_id']
    source_id     = ds.attrs['source_id']
    #regrid
    regridder = xe.Regridder(ds, ds_out, 'nearest_s2d', reuse_weights=False)
    ds_new    = regridder(ds)
    
#     #Assign back attributes as regirdder would have deleted attributes 
    ds_new.attrs['experiment_id'] = experiment_id
    ds_new.attrs['source_id'] = source_id
    
    #print(ds_new.attrs['experiment_id'],ds_new.attrs['source_id'])
    #print(ds_new)
    return ds_new

def process_data(ds, quantiles=quants):
    ds = ds.pipe(drop_feb29).pipe(to_daily).pipe(extract_data)
    
    if len(ds['year']) == 0:
        print("The dataset is empty. Skipping...")
        return None
    
    if len(ds['dayofyear'])<365:
        print('The dataset has less than 365 days. Skipping ..')
        return None
    
    # # Remove 'time' coordinate
    # ds = ds.set_index(time=("year", "dayofyear")).unstack("time")  
   
    
    return (ds.pipe(regrid, ds_out=ds_out)
           )


In [15]:
# with progress.ProgressBar():

#     expt_da = xr.DataArray(expts, dims='experiment_id', name='experiment_id',
#                            coords={'experiment_id': expts})

#     # Initialize an Empty Dictionary for Aligned Datasets:
#     dsets_aligned = {}

#     # Iterate Over dsets_ Dictionary:

#     for k, v in tqdm(dsets_.items()):
#         # Initialize a dictionary for this source_id
#         dsets_aligned[k] = {}
        
#         skip_source_id = False

#         for expt in expts:
#             ds = v[expt].pipe(process_data)

#             # Check if the dataset is empty and skip this source_id if so
#             if ds is None:
#                 print(f"Skipping {expt} for {k} because the dataset is empty")
#                 skip_source_id = True
#                 break
            
#             # Store the dataset in the dictionary
#             # dsets_aligned[k][expt] = ds
#             # Compute the dataset and store it in the dictionary
#             dsets_aligned[k][expt] = ds.compute()
#             print(dsets_aligned[k][expt])

#         if skip_source_id:
#             del dsets_aligned[k]
#             continue

In [16]:
# with progress.ProgressBar():
#     dsets_aligned_ = dask.compute(dsets_aligned)[0]

In [17]:
# source_ids = list(dsets_aligned.keys())
# source_da = xr.DataArray(source_ids, dims='source_id', name='source_id',
#                          coords={'source_id': source_ids})

# # final_ds = {expt: xr.concat([ds.get(expt, xr.Dataset()).reset_coords(drop=True)
# #                              for ds in dsets_aligned.values()],
# #                             dim=source_da)
# #             for expt in expts}

# final_ds = {expt: xr.concat([ds[expt].reset_coords(drop=True)
#                              for ds in dsets_aligned.values() if expt in ds and ds[expt] is not None],
#                             dim=source_da, coords='minimal')
#             for expt in expts}

# final_ds

In [18]:
# final_ds_pi = xr.concat([ds['historical'].reset_coords(drop=True)
#                                  for ds in dsets_aligned.values()],
#                                 dim=source_da)

# final_ds_eoc = xr.concat([ds['ssp370'].reset_coords(drop=True)
#                              for ds in dsets_aligned.values()],
#                             dim=source_da)
# final_ds_eoc

In [19]:
%%time
# final_ds_pi.to_dataset().to_zarr(cvals+'cmip6_pi_quantiles_annual.zarr',mode='w')
# final_ds_eoc.to_dataset().to_zarr(cvals+'cmip6_eoc_quantiles_annual.zarr',mode='w')
# final_ds_pi.to_zarr(cvals+'cmip6_pi_1.5grid_quantiles_annual.zarr',mode='w')
# final_ds_eoc.to_zarr(cvals+'cmip6_eoc_1.5grid_quantiles_annual.zarr',mode='w')

CPU times: user 17 µs, sys: 0 ns, total: 17 µs
Wall time: 42.7 µs


In [20]:
final_ds_pi  = xr.open_zarr(cvals+'cmip6_pi_quantiles_annual.zarr').tasmax
final_ds_eoc = xr.open_zarr(cvals+'cmip6_eoc_quantiles_annual.zarr').tasmax
# final_ds_pi  = xr.open_zarr(cvals+'cmip6_pi_1.5grid_quantiles_annual.zarr').tasmax
# final_ds_eoc = xr.open_zarr(cvals+'cmip6_eoc_1.5grid_quantiles_annual.zarr').tasmax
final_ds_eoc

<xarray.DataArray 'tasmax' (source_id: 18, year: 30, dayofyear: 365, lat: 61,
                            lon: 121)>
dask.array<open_dataset-7b83895101ae604b25277a61bd350bc4tasmax, shape=(18, 30, 365, 61, 121), dtype=float32, chunksize=(3, 8, 92, 16, 31), chunktype=numpy.ndarray>
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
  * source_id  (source_id) <U17 'ACCESS-CM2' 'AWI-CM-1-1-MR' ... 'NorESM2-MM'
  * year       (year) int64 2071 2072 2073 2074 2075 ... 2097 2098 2099 2100

### Detrend data and save

In [21]:
%%time
ds_pi_det  = detrend_data(final_ds_pi,pi_year)
ds_eoc_det = detrend_data(final_ds_eoc,eoc_year)
ds_eoc_det

CPU times: user 7min 20s, sys: 10min 51s, total: 18min 11s
Wall time: 14min 12s


<xarray.DataArray (source_id: 18, year: 30, dayofyear: 365, lat: 61, lon: 121)>
dask.array<sub, shape=(18, 30, 365, 61, 121), dtype=float64, chunksize=(1, 8, 92, 16, 31), chunktype=numpy.ndarray>
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
  * source_id  (source_id) <U17 'ACCESS-CM2' 'AWI-CM-1-1-MR' ... 'NorESM2-MM'
  * year       (year) int64 2071 2072 2073 2074 2075 ... 2097 2098 2099 2100
    degree     int64 1

In [22]:
# %%time
# pcoeffs_ds_pi  = final_ds_pi.polyfit(dim='year',deg=1,skipna=False)
# pcoeffs_ds_eoc = final_ds_eoc.polyfit(dim='year',deg=1,skipna=False)
# pcoeffs_ds_eoc

In [23]:
# %%time
# m_pi  = pcoeffs_ds_pi.polyfit_coefficients.sel(degree=1)
# m_eoc = pcoeffs_ds_eoc.polyfit_coefficients.sel(degree=1)
# m_pi

In [24]:
# %%time
# #Save slopes
# m_pi.to_dataset().to_zarr(cvals+'cmip6_pi_trend.zarr',mode='w')
# m_eoc.to_dataset().to_zarr(cvals+'cmip6_eoc_trend.zarr',mode='w')

In [25]:
# m_pi  = xr.open_zarr(cvals+'cmip6_pi_trend.zarr').polyfit_coefficients
# m_eoc = xr.open_zarr(cvals+'cmip6_eoc_trend.zarr').polyfit_coefficients

In [26]:
# #Slopes for Chicago:
# print(m_eoc.sel(dayofyear=365).sel(lat=chicago_lat,lon=chicago_lon,method='nearest').values)

In [27]:
# print(m_pi.sel(dayofyear=365).sel(lat=chicago_lat,lon=chicago_lon,method='nearest').values)

In [28]:
# #Calculate trend
# pi_trend   = m_pi*(final_ds_pi['year']  - pi_year)
# eoc_trend  = m_eoc*(final_ds_eoc['year']- eoc_year)
# #Subtract trend
# ds_pi_det  = final_ds_pi  - pi_trend
# ds_eoc_det = final_ds_eoc - eoc_trend

In [29]:
# #
# ds_pi_det  = ds_pi_det.rename('tasmax')
# ds_eoc_det = ds_eoc_det.rename('tasmax')
# ds_pi_det

In [30]:
# %%time
# ds_eoc_det.sel(dayofyear=365).sel(lat=chicago_lat,lon=chicago_lon,method='nearest').sel(source_id='AWI-CM-1-1-MR').values

In [31]:
# ds_eoc_det.sel(dayofyear=365).sel(lat=chicago_lat,lon=chicago_lon,method='nearest').sel(source_id='AWI-CM-1-1-MR').values

In [32]:
# final_ds_eoc.sel(dayofyear=365).sel(lat=chicago_lat,lon=chicago_lon,method='nearest').sel(source_id='AWI-CM-1-1-MR').values

In [33]:
ds_pi_det['source_id']  = ds_pi_det['source_id'].astype(str)
ds_eoc_det['source_id'] = ds_eoc_det['source_id'].astype(str)

In [34]:
# %%time
# ds_pi_det.to_dataset().to_zarr(cvals+'cmip6_pi_ann_detrended.zarr',mode='w')
# ds_eoc_det.to_dataset().to_zarr(cvals+'cmip6_eoc_ann_detrended.zarr',mode='w')
# ds_pi_det.to_dataset().to_zarr(cvals+'cmip6_pi_ann_1.5grid_detrended.zarr',mode='w')
# ds_eoc_det.to_dataset().to_zarr(cvals+'cmip6_eoc_ann_1.5grid_detrended.zarr',mode='w')

### Compute global and annual mean

In [35]:
ds_pi_det  = xr.open_zarr(cvals+'cmip6_pi_ann_detrended.zarr').tasmax
ds_eoc_det = xr.open_zarr(cvals+'cmip6_eoc_ann_detrended.zarr').tasmax
# ds_pi_det  = xr.open_zarr(cvals+'cmip6_pi_ann_1.5grid_detrended.zarr').tasmax
# ds_eoc_det = xr.open_zarr(cvals+'cmip6_eoc_ann_1.5grid_detrended.zarr').tasmax
#
qpi  = compute_quantiles(ds_pi_det)
qeoc = compute_quantiles(ds_eoc_det)
qano = qeoc - qpi
qano

<xarray.DataArray 'tasmax' (quantile: 30, source_id: 18, dayofyear: 365,
                            lat: 61, lon: 121)>
dask.array<sub, shape=(30, 18, 365, 61, 121), dtype=float64, chunksize=(30, 1, 92, 16, 31), chunktype=numpy.ndarray>
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
  * source_id  (source_id) <U17 'ACCESS-CM2' 'AWI-CM-1-1-MR' ... 'NorESM2-MM'
  * quantile   (quantile) float64 0.0 0.03448 0.06897 ... 0.931 0.9655 1.0

In [36]:
# %%time
# test = qano.sel(dayofyear=365).sel(lat=chicago_lat,lon=chicago_lon,method='nearest').\
# sel(quantile=0.03448,method='nearest').values

CPU times: user 6.35 s, sys: 410 ms, total: 6.76 s
Wall time: 11.6 s


In [37]:
# test

array([ 4.97339247,  3.19495899,  5.06688141, 14.98609247,  3.73884816,
        9.55674148, 13.49184208, 13.22518559,  6.71714727,  2.42355726,
        3.73900106,  9.757262  ,  3.81634905, 11.50711233,  8.04899877,
        3.46947516,  4.20171598,  3.87958733])

In [38]:
# test.std()

3.9791766198591207

In [39]:
%%time
qano_std            = qano.std(dim='source_id')
qano_cmip_mean      = qano.mean(dim='source_id')
qano_sq_deviation   = (qano - qano_cmip_mean)**2
qano_sq_deviation

CPU times: user 264 ms, sys: 18.7 ms, total: 283 ms
Wall time: 287 ms


<xarray.DataArray 'tasmax' (quantile: 30, source_id: 18, dayofyear: 365,
                            lat: 61, lon: 121)>
dask.array<pow, shape=(30, 18, 365, 61, 121), dtype=float64, chunksize=(30, 1, 92, 16, 31), chunktype=numpy.ndarray>
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
  * source_id  (source_id) <U17 'ACCESS-CM2' 'AWI-CM-1-1-MR' ... 'NorESM2-MM'
  * quantile   (quantile) float64 0.0 0.03448 0.06897 ... 0.931 0.9655 1.0

In [ ]:
%%time 
qano_mae = global_mean(np.abs(qano))
qano_mae = qano_mae.rename('tmax')
qano_mae

In [ ]:
qano_mae.to_dataset().to_zarr(cvals + 'cmip6_absano_agmean.zarr')

In [41]:
qano_std_agmean  = global_mean(qano_std)
qano_msd         = global_mean(qano_sq_deviation)
qano_rmsd        = np.sqrt(qano_msd)
qano_rmsd

<xarray.DataArray (quantile: 30)>
dask.array<sqrt, shape=(30,), dtype=float64, chunksize=(30,), chunktype=numpy.ndarray>
Coordinates:
  * quantile  (quantile) float64 0.0 0.03448 0.06897 0.1034 ... 0.931 0.9655 1.0

In [42]:
# %%time
# qano_std.sel(dayofyear=365).sel(lat=chicago_lat,lon=chicago_lon,method='nearest').values

In [43]:
# %%time
# qano_std.sel(dayofyear=365).sel(lat=1,lon=5,method='nearest').values

In [44]:
%%time
qano_std_agmean = qano_std_agmean.rename('tasmax_qanomaly')
qano_rmsd       = qano_rmsd.rename('qanomaly_rmsd')
# qano_std_agmean.to_dataset().to_zarr(cvals+'cmip6_quantiles_ano_agmean.zarr',mode='w')
# qano_rmsd.to_dataset().to_zarr(cvals+'cmip6_qano_ag_rmsd.zarr',mode='w')
# qano_std_agmean.to_dataset().to_zarr(cvals+'cmip6_quantiles_ano_1.5grid_agmean.zarr',mode='w')
# qano_rmsd.to_dataset().to_zarr(cvals+'cmip6_qano_ag_1.5grid_rmsd.zarr',mode='w')

CPU times: user 132 µs, sys: 0 ns, total: 132 µs
Wall time: 182 µs


In [45]:
qano_std_agmean = xr.open_zarr(cvals+'cmip6_quantiles_ano_1.5grid_agmean.zarr').tasmax_qanomaly
qano_std_agmean.values

FileNotFoundError: No such file or directory: '/global/scratch/users/harsha/LENS/cesm2/cvals/detrended/cmip6_quantiles_ano_1.5grid_agmean.zarr'

In [ ]:
qano_std_agmean = xr.open_zarr(cvals+'cmip6_quantiles_ano_agmean.zarr').tasmax_qanomaly
qano_std_agmean.values

In [ ]:
qano_rmsd = xr.open_zarr(cvals+'cmip6_qano_ag_rmsd.zarr').qanomaly_rmsd
qano_rmsd.values

In [ ]:
qano_std_agmean.plot()
plt.xlabel('Probability p')
plt.ylabel('Global, annual mean of standard deviation (K) ')
plt.title(r'Global, annual mean of stdev in $Q_f(p) -Q_i(p)$ among 18 CMIP6 models')
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.05, 0.95, r'18 models, $1.5^{\circ} \times 1.5^{\circ}$ grid', transform=plt.gca().transAxes, fontsize=14,
        verticalalignment='top', bbox=props)
plt.ylim(0,1.8)
plt.xlim(0,1.0)

In [ ]:
qano_std_agmean.plot()
plt.xlabel('Probability p')
plt.ylabel('Global, annual mean of standard deviation (K) ')
plt.title(r'Global, annual mean of stdev in $Q_f(p) -Q_i(p)$ among 18 CMIP6 models')
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.05, 0.95, r'18 models, $3^{\circ} \times 3^{\circ}$ grid', transform=plt.gca().transAxes, fontsize=14,
        verticalalignment='top', bbox=props)
plt.ylim(0,1.8)
plt.xlim(0,1.0)

In [ ]:
qano_rmsd.plot()
plt.xlabel('Probability p')
plt.ylabel('RMSD (K) ')
plt.title(r'RMSD of $Q_f(p) -Q_i(p)$')
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.05, 0.95, r'18 models, $3^{\circ} \times 3^{\circ}$ grid', transform=plt.gca().transAxes, fontsize=14,
        verticalalignment='top', bbox=props)
plt.ylim(0,2.5)
plt.xlim(0,1.0)